# Populate the portal using the Wikibase API
Goal of this notebook: Import the data prepared in `filter_papers_by_software.ipynb` into the data structure setup in `import_wikidata_properties.ipynb`.

The API documentation is [here](https://www.wikidata.org/w/api.php?action=help&modules=wbeditentity)

In [2]:
# import common definitions and functions
%run WB_common.ipynb

## Login to the Wikibase

### Network settings
* Make sure the wikibase is running, e.g. using [MaRDI4NFDI/portal-compose](https://github.com/MaRDI4NFDI/portal-compose)
* Make sure this jupyter notebook is in the same network as the wiki. This is done in docker-compose

```
networks:
  default:
    external: true
    name: portal-compose_default
```

Networks can be listed using `docker network ls`. Here, "portal-compose_default" is the name of the network started by portal-compose.
* Verify that this notebook is in the correct network `docker network inspect portal-compose_default`

The wiki is then accessible from the notebook container at `http://mardi-wikibase`.

In [3]:
import requests
import json 
import configparser

# url of the API endpoint
WIKIBASE_API = 'http://mardi-wikibase/w/api.php?format=json'

def login(username, botpwd):
    """
    Starts a new session and logins using a bot account.
    @username, @botpwd string: credentials of an existing bot user
    @returns requests.sessions.Session object
    """
    # create a new session
    session = requests.Session()

    # get login token
    r1 = session.get(WIKIBASE_API, params={
        'format': 'json',
        'action': 'query',
        'meta': 'tokens',
        'type': 'login'
    })
    # login with bot account
    r2 = session.post(WIKIBASE_API, data={
        'format': 'json',
        'action': 'login',
        'lgname': username,
        'lgpassword': botpwd,
        'lgtoken': r1.json()['query']['tokens']['logintoken'],
    })
    # raise when login failed
    if r2.json()['login']['result'] != 'Success':
        raise WBAPIException(r2.json()['login'])
        
    return session

### Bot user
* Login to the wiki as admin
* Go to Special:BotPasswords, create a bot user, call it "import", grant it "High-volume editing", "Edit existing pages", "Create, edit, and move pages"
* Copy `data/credentials.tpl` to `data/credentials.ini`. Replace the username and password by those of the newly created bot user (make sure not to commit this file)

In [104]:
# read bot username and password from data/credentials.ini
config = configparser.ConfigParser()
config.sections()
config.read('data/credentials.ini')
username = config['default']['username']
botpwd = config['default']['password']

session = login(username, botpwd)

## Create a wikibase property
A function that creates a new wikidatabase property and returns the new id.

If the property label already exists in the wiki, will not overwrite it, but raise an error.

In [6]:
def get_csrf_token(session):
    """Gets a security (CSRF) token."""
    params1 = {
        "action": "query",
        "meta": "tokens",
        "type": "csrf"
    }
    r1 = session.get(WIKIBASE_API, params=params1)
    token = r1.json()['query']['tokens']['csrftoken']

    return token
    

def create_property(session, data):
    """
    Creates a wikibase property.
    @session requests.sessions.Session: session obtained from login 
    @data python dict: creation parameters of the property
    @returns string: id of the new property
    """
    token = get_csrf_token(session)
    
    params = {
        "action": "wbeditentity",
        "format": "json",
        'new': 'property',
        'data': json.dumps(data),
        'token': token
    }
    r1 = session.post(WIKIBASE_API, data=params)
    r1.json = r1.json()
    
    # raise when edit failed
    if 'error' in r1.json.keys():
        raise WBAPIException(r1.json['error'])

    return r1.json['entity']['id']

For example create a property with these parameters will return an id-string in the form 'Px' (where x is a number).

The property can be seen in the wiki under `http:localhost:8080/wiki/Property:Px`

In [5]:
data = {"labels":{"en":{"language":"en","value":"Propertylabel9"}},"descriptions":{"en":{"language":"en","value":"Propertydescription"}},"datatype":"string"}
create_property(session, data)

'P1'

## Create a wikibase entity
A function that creates a new wikidatabase entity (item) and returns the new id.

If the entity label already exists in the wiki, will not overwrite it, but create a new entity.

In [7]:
def create_entity(session, data):
    """
    Creates a wikibase entity.
    @session requests.sessions.Session: session obtained from login 
    @data python dict: creation parameters of the entity
    @returns string: id of the new entity
    """
    token = get_csrf_token(session)
    
    params = {
        'action': 'wbeditentity',
        'format': 'json',
        'new': 'item',
        'data': json.dumps(data),
        'token': token
    }
    r1 = session.post(WIKIBASE_API, data=params)
    r1.json = r1.json()
    
    # raise when edit failed
    if 'error' in r1.json.keys():
        raise WBAPIException(r1.json['error'])

    return r1.json['entity']['id']

For example create an item with these parameters will return an id-string in the form 'Qx' (where x is a number).

The item can be seen in the wiki under `http:localhost:8080/wiki/Item:Qx`

In [7]:
data={"labels":{"de":{"language":"de","value":"de-value"},"en":{"language":"en","value":"en-value"}}}
create_entity(session, data)

'Q320'

## Import the authors list
Before importing anything, make sure the corresponding items and properties have been imported from wikidata. See notebook `WB_wikidata_properties.ipynb`.

A subsample of the authors list was created in notebook `filter_papers_by_software.ipyb`. This list contains the authors of a papers related to the first 1000 software entries in the list of softwares (`data/swMath-software-list.csv`). The list of authors is in file `data/all_authors.csv.zip`. 

In [87]:
# load the list of authors
import pandas as pd

# load the list of zbMath authors
authors_df = pd.read_csv('data/all_authors.csv.zip') 
authors_df.head()

,author_id,author_name
0,aidun.cyrus-k,"Aidun, Cyrus K."
1,babuska.ivo-m,"Babuška, I."
2,banerjee.uday,"Banerjee, U."
3,bartels.alexander,"Bartels, Alexander"
4,basko.mikhail-m,"Basko, M. M."


Use the create_entity function to import the authors into the wiki.
The Q-id returned by the wikibase is appended to the pandas dataframe of authors.

In [91]:
for i,current in authors_df.iterrows():
    author_name = current['author_name'].strip()
    author_id = current['author_id'].strip()
    data = {
        'labels':{'en':{'language':'en','value':author_name}},
        'claims': [
            # instance of 'human'
            {'mainsnak':{
                'snaktype':'value', 'property':'P31', 'datavalue':{'type':'wikibase-entityid', 'value': {'entity-type':'item','id':'Q5'}}},
            'type': 'statement', 'rank': 'normal'},
            # zbMath author id
            {'mainsnak':{
                'snaktype':'value', 'property':'P1556', 'datavalue': {'type':'string', 'value': author_id}},
            'type': 'statement', 'rank': 'normal'}
            ]
    }
    # import into wikibase, save Qid
    authors_df.loc[i, 'qid'] = create_entity(session, data)

In [92]:
authors_df.head()

,author_id,author_name,qid
0,aidun.cyrus-k,"Aidun, Cyrus K.",Q1104
1,babuska.ivo-m,"Babuška, I.",Q1105
2,banerjee.uday,"Banerjee, U.",Q1106
3,bartels.alexander,"Bartels, Alexander",Q1107
4,basko.mikhail-m,"Basko, M. M.",Q1108


## Import the software list
All software entries have already been imported into the MaRDI portal.
Here I will import the first 10 (out of 40000) software entries into the local wiki for testing.

In [37]:
MAX_ENTRIES = 10 # number of software entries to process

# load the list of swMath software
software_df = pd.read_csv('data/swMATH-software-list.csv')
software_df = software_df[:MAX_ENTRIES]
software_df.head()

,qid,P13,Len,#
0,NaN,'0',swMATH,initial csv import 2021-12-17
1,NaN,'1',FORTRAN,initial csv import 2021-12-17
2,NaN,'2',SuperLU-DIST,initial csv import 2021-12-17
3,NaN,'3',WHISPAR,initial csv import 2021-12-17
4,NaN,'4',MULTI2D,initial csv import 2021-12-17


Use the create_entity function to import the software into the wiki.
The Q-id returned by the wikibase is appended to the pandas dataframe of software.

In [41]:
for i,current in software_df[:MAX_ENTRIES].iterrows():
    software_name = current['Len'].strip()
    software_id = current['P13'].strip().replace("'", '')

    data = {
        'labels':{'en':{'language':'en','value':software_name}},
        'claims': [
            # instance of 'software'
            {'mainsnak':{
                'snaktype':'value', 'property':'P31', 'datavalue':{'type':'wikibase-entityid', 'value': {'entity-type':'item','id':'Q7397'}}},
            'type': 'statement', 'rank': 'normal'},
            # swMath work id
            {'mainsnak':{
                'snaktype':'value', 'property':'P6830', 'datavalue': {'type':'string', 'value': software_id}},
            'type': 'statement', 'rank': 'normal'}
            ]
    }
    # import into wikibase, save Qid
    software_df.loc[i, 'qid'] = create_entity(session, data)

In [42]:
software_df.head()

,qid,P13,Len,#
0,Q1034,'0',swMATH,initial csv import 2021-12-17
1,Q1035,'1',FORTRAN,initial csv import 2021-12-17
2,Q1036,'2',SuperLU-DIST,initial csv import 2021-12-17
3,Q1037,'3',WHISPAR,initial csv import 2021-12-17
4,Q1038,'4',MULTI2D,initial csv import 2021-12-17


## Import the papers list
A subsample of the papers list was created in notebook `filter_papers_by_software.ipyb`. This list contains the papers related to the first 10 software entries in the list of softwares (`data/swMath-software-list.csv`). The list of papers is in file `data/all_papers.csv.zip`. 

In [43]:
# load the list of zbMath papers
papers_df = pd.read_csv('data/all_papers.csv.zip')
papers_df.head(3)

,id,author,author_ids,document_title,source,classifications,language,links,keywords,publication_year,doi
0,oai:zbmath.org:5181224,"Zlatev, Zahari; Dimov, Ivan",zlatev.zahari;dimov.ivan-todor,Computational and numerical challenges in envi...,Studies in Computational Mathematics 13. Amste...,65M20;65-02;65Y05;35Kxx,English,http://www.sciencedirect.com/science/book/9780...,textbook;parallel computation;semidiscretizati...,2006,NaN
1,oai:zbmath.org:5187737,"Buttari, Alfredo; D'Ambra, Pasqua; di Serafino...",buttari.alfredo;dambra.pasqua;di-serafino.dani...,2LEV-D2P4: a package of high-performance preco...,"Appl. Algebra Eng. Commun. Comput. 18, No. 3, ...",65F35;65F10;65Y15;65Y05,English,NaN,parallel numerical software;algebraic two-leve...,2007,10.1007/s00200-007-0035-z
2,oai:zbmath.org:5187739,"Gupta, Anshul",gupta.anshul,A shared- and distributed-memory parallel gene...,"Appl. Algebra Eng. Commun. Comput. 18, No. 3, ...",65F05;65F50;68W30;65Y05,English,NaN,sparse matrix factorization;parallel sparse so...,2007,10.1007/s00200-007-0037-x


In [ ]:
for i,current in papers_df.iterrows():
    document_title = current['document_title'].strip()
    document_id = current['id'].strip()
    publication_date = "+{}-01-01T00:00:00Z".format(current['publication_year'])
    
    data = {
        'labels':{'en':{'language':'en','value':document_title}},
        'claims': [
            # instance of 'scholarly article'
            {'mainsnak':{
                'snaktype':'value', 'property':'P31', 'datavalue':{'type':'wikibase-entityid', 'value': {'entity-type':'item','id':'Q13442814'}}},
            'type': 'statement', 'rank': 'normal'},
            # zbMath work id
            {'mainsnak':{
                'snaktype':'value', 'property':'P894', 'datavalue': {'type':'string', 'value': document_id}},
            'type': 'statement', 'rank': 'normal'},
            # publication date
            {'mainsnak':{
                'snaktype':'value', 'property':'P577', 'datavalue': {'type':'time', 'value': { 
                    "time": publication_date, 'precision': 9, 'timezone':0, 'before':0, 'after':0, 'calendarmodel':'http://www.wikidata.org/entity/Q1985727'
                }}},
            'type': 'statement', 'rank': 'normal'}
            ]
    }
    # import into wikibase, save qid
    papers_df.loc[i, 'qid'] = create_entity(session, data)

In [102]:
from datetime import datetime
# "+1838-01-01T00:00:00Z"
datetime.strftime(datetime.strptime('2006', '%Y'))

TypeError: strftime() missing required argument 'format' (pos 1)

### Append the paper-to-software relations
**Papers may use multiple softwares**. The relation between papers and software is in an additional file `data/all_papers_software.csv.zip`.

In [85]:
# load the list of papers-to-software relations
papers_software_df = pd.read_csv('data/all_papers_software.csv.zip')
papers_software_df.head(3)

,id,software
0,oai:zbmath.org:5181224,SuperLU-DIST
1,oai:zbmath.org:5187737,SuperLU-DIST
2,oai:zbmath.org:5187739,SuperLU-DIST


### Edit a wikibase entity

In [ ]:
def edit_entity(session, qid, data):
    token = get_csrf_token(session)
    
    params = {
        'id': qid,
        'action': 'wbeditentity',
        'format': 'json',
        'data': json.dumps(data),
        'token': token
    }
    r1 = session.post(WIKIBASE_API, data=params)
    r1.json = r1.json()
    
    # raise when edit failed
    if 'error' in r1.json.keys():
        raise WBAPIException(r1.json['error'])

    return r1.json['entity']['id']


In [86]:
groups = papers_software_df.groupby('id')
for paper_ref,group in groups:
    paper_id = papers_df[papers_df['id'] == paper_ref]['qid'].values[0]
    for software_name in group['software']:
        software_id = software_df[software_df['Len'] == software_name]['qid']
        software_id = software_id.values[0]
        data = {
            'claims': [
                {
                    # describe project that uses
                    'mainsnak':{
                        'snaktype':'value', 'property':'P4510', 'datavalue':{'type':'wikibase-entityid', 'value': {'entity-type':'item','id':software_id}}},
                        'type': 'statement', 'rank': 'normal'}]
        }
        edit_entity(session, paper_id, data)

### Append paper to author relations

In [84]:
# load the list of papers to authors relations
papers_authors_df = pd.read_csv('data/all_papers_authors.csv.zip')
papers_authors_df.head(3)

,Unnamed: 0,paper_id,author_id
0,0,oai:zbmath.org:5181224,zlatev.zahari
1,1,oai:zbmath.org:5181224,dimov.ivan-todor
2,2,oai:zbmath.org:5187737,buttari.alfredo


In [97]:
groups = papers_authors_df.groupby('paper_id')
for paper_ref,group in groups:
    paper_id = papers_df[papers_df['id'] == paper_ref]['qid'].values[0]
    for author_ref in group['author_id']:
        author_qid = authors_df[authors_df['author_id'] == author_ref]['qid']
        author_qid = author_qid.values[0]
        data = {
            'claims': [
                {
                    # paper has author
                    'mainsnak':{
                        'snaktype':'value', 'property':'P50', 'datavalue':{'type':'wikibase-entityid', 'value': {'entity-type':'item','id':author_qid}}},
                        'type': 'statement', 'rank': 'normal'}]
        }
        edit_entity(session, paper_id, data)

In [93]:
authors_df.head()

,author_id,author_name,qid
0,aidun.cyrus-k,"Aidun, Cyrus K.",Q1104
1,babuska.ivo-m,"Babuška, I.",Q1105
2,banerjee.uday,"Banerjee, U.",Q1106
3,bartels.alexander,"Bartels, Alexander",Q1107
4,basko.mikhail-m,"Basko, M. M.",Q1108
